In [12]:
import os
import pandas as pd

def get_taq_link(db, permno_list=None, year=None, start_year=2000):
    """
    Get TAQ linking table to link daily TAQ to CRSP.

    Parameters
    ----------
    db : object
        WRDS connection with a .raw_sql() method.
    permno_list : list[int] | None
        PERMNOs to filter on. If None, returns all PERMNOs.
    year : int | 'all' | None
        If int, restricts to that calendar year (YYYY).
        If 'all' or None, no single-year filter is applied.
    start_year : int | None
        If set, restricts to dates >= January 1 of start_year.

    Returns
    -------
    pandas.DataFrame
        Columns (typical): permno, date, cusip, sym_root, sym_suffix
    """
    # Ensure required directories exist (optional)
    os.makedirs('data', exist_ok=True)
    os.makedirs('data/crsp', exist_ok=True)

    # Build predicates and parameters safely
    where_clauses = []
    params = {}

    if permno_list:
        # Make a list of positional parameters: %(p0)s, %(p1)s, ...
        ph = []
        for i, p in enumerate(permno_list):
            key = f"p{i}"
            params[key] = int(p)  # ensure int
            ph.append(f"%({key})s")
        where_clauses.append(f"a.permno IN ({', '.join(ph)})")

    if isinstance(year, int):
        params["y_start"] = f"{year}-01-01"
        params["y_end"]   = f"{year}-12-31"
        where_clauses.append("a.date BETWEEN %(y_start)s AND %(y_end)s")

    if start_year is not None and not isinstance(year, int):
        # Only apply start_year if you didn't already pin to a specific year
        params["start_year"] = f"{start_year}-01-01"
        where_clauses.append("a.date >= %(start_year)s")

    # Default WHERE to true if no filters provided
    where_sql = " AND ".join(where_clauses) if where_clauses else "1=1"

    sql = f"""
        SELECT
            a.permno,
            a.date,
            a.cusip,
            a.sym_root,
            a.sym_suffix
        FROM wrdsapps.taqmclink a
        WHERE
            a.date >= DATE '2024-01-01'
        ORDER BY a.date;
    """

    df = db.raw_sql(sql, params=params)
    return df


In [13]:
import os
import dotenv
import pandas as pd

import wrds

def connect_wrds(username, password):
    print(f"Connecting to WRDS with username: {username}")
    db = wrds.Connection(wrds_username=username, wrds_password=password, use_keyring=False)
    print("Connected to WRDS!")
    return db


dotenv.load_dotenv()


# hyperparameters
FACTOR_PATH = 'data/factors'

# connect to db
db = connect_wrds(username=os.getenv("WRDS_USERNAME"), password=os.getenv("WRDS_PASSWORD"))

Connecting to WRDS with username: andrekraemer
Loading library list...
Done
Connected to WRDS!


In [14]:
taq_link = get_taq_link(db)
taq_link.head()

,permno,date,cusip,sym_root,sym_suffix
0,22563,2024-01-02,45827K10,INTE,<NA>
1,92220,2024-01-02,57479510,MASI,<NA>
2,24090,2024-01-02,25365120,DBD,<NA>
3,84319,2024-01-02,53679710,LAD,<NA>
4,89900,2024-01-02,87162W10,SNX,<NA>


In [52]:
# Change date to datetime
taq_link['date'] = pd.to_datetime(taq_link['date'], format='%Y-%m-%d', errors='coerce')

In [59]:
# Load TAQ data
df = pd.read_csv("data/taqtrades2024.csv")
print(len(df))
df.head()

2426538


,date,SYM_ROOT,SYM_SUFFIX,nb,ns,sb,ss,vb,vs
0,02JAN2024,A,NaN,719,644,31347,27518,4.361987e+06,3.825107e+06
1,02JAN2024,AA,NaN,1379,1387,206068,202722,6.905840e+06,6.791301e+06
2,02JAN2024,AAA,NaN,3,5,442,153,1.107547e+04,3.825193e+03
3,02JAN2024,AAAU,NaN,256,159,189555,216372,3.872967e+06,4.421058e+06
4,02JAN2024,AACG,NaN,16,12,1539,1325,1.711869e+03,1.437439e+03


In [ ]:
# No empty rows
num_rows_all_na = df.isna().all(axis=1).sum()
print(num_rows_all_na)

0


In [71]:
# Convert TAQ date
df['date'] = pd.to_datetime(df['date'], format='%d%b%Y', errors='coerce')

# Change column names
df.rename(columns={'SYM_ROOT': 'sym_root', 'SYM_SUFFIX': 'sym_suffix'}, inplace=True)
df.head()


,date,sym_root,sym_suffix,nb,ns,sb,ss,vb,vs
0,2024-01-02,A,NaN,719,644,31347,27518,4.361987e+06,3.825107e+06
1,2024-01-02,AA,NaN,1379,1387,206068,202722,6.905840e+06,6.791301e+06
2,2024-01-02,AAA,NaN,3,5,442,153,1.107547e+04,3.825193e+03
3,2024-01-02,AAAU,NaN,256,159,189555,216372,3.872967e+06,4.421058e+06
4,2024-01-02,AACG,NaN,16,12,1539,1325,1.711869e+03,1.437439e+03


In [ ]:
# Link TAQ with CRSP link table
linked_table = df.merge(
    taq_link[['date', 'sym_root', 'sym_suffix', 'permno']],
    on=['date', 'sym_root', 'sym_suffix'],
    how='inner',
    validate='many_to_many'
)

In [ ]:
# Get number of rows left
print(len(linked_table))
linked_table.head(10)

1538287


,date,sym_root,sym_suffix,nb,ns,sb,ss,vb,vs,permno
0,2024-01-02,A,NaN,719,644,31347,27518,4.361987e+06,3.825107e+06,87432
1,2024-01-02,AA,NaN,1379,1387,206068,202722,6.905840e+06,6.791301e+06,16347
2,2024-01-02,AAA,NaN,3,5,442,153,1.107547e+04,3.825193e+03,23483
3,2024-01-02,AACI,NaN,2,0,100,0,1.079950e+03,0.000000e+00,22271
4,2024-01-02,AADI,NaN,113,131,26505,47382,5.497834e+04,9.727473e+04,17869
5,2024-01-02,AADR,NaN,4,4,29,121,1.598490e+03,6.644095e+03,10113
6,2024-01-02,AAGR,NaN,114,57,39412,26991,4.304845e+04,2.827992e+04,22253
7,2024-01-02,AAL,NaN,5086,3881,2956421,2128543,4.003742e+07,2.883191e+07,21020
8,2024-01-02,AAMC,NaN,7,10,42,159,1.734055e+02,6.432273e+02,14127
9,2024-01-02,AAME,NaN,16,9,4042,2114,9.835415e+03,5.086436e+03,15580


In [ ]:
# Sanity Check. ETFs are excluded
print('AAAU' in df['sym_root'].values)
print('AAAU' in taq_link['sym_root'].values)
print('AAAU' in linked_table['sym_root'].values)

True
False
False


In [ ]:
##############################################

# Now: Get IBES data

##############################################

In [19]:
# Load IBES data
import os
import pandas as pd

def get_ibes(db, permno_list=None, year=None, start_year=2000):
    """
    Get IBES price target data.

    Parameters
    ----------
    db : object
        WRDS connection with a .raw_sql() method.
    permno_list : list[int] | None
        PERMNOs to filter on. If None, returns all PERMNOs.
    year : int | 'all' | None
        If int, restricts to that calendar year (YYYY).
        If 'all' or None, no single-year filter is applied.
    start_year : int | None
        If set, restricts to dates >= January 1 of start_year.

    Returns
    -------
    pandas.DataFrame
        Columns (typical):
    """
    # Ensure required directories exist (optional)
    os.makedirs('data', exist_ok=True)
    os.makedirs('data/crsp', exist_ok=True)

    # Build predicates and parameters safely
    where_clauses = []
    params = {}

    if permno_list:
        # Make a list of positional parameters: %(p0)s, %(p1)s, ...
        ph = []
        for i, p in enumerate(permno_list):
            key = f"p{i}"
            params[key] = int(p)  # ensure int
            ph.append(f"%({key})s")
        where_clauses.append(f"a.permno IN ({', '.join(ph)})")

    if isinstance(year, int):
        params["y_start"] = f"{year}-01-01"
        params["y_end"]   = f"{year}-12-31"
        where_clauses.append("a.date BETWEEN %(y_start)s AND %(y_end)s")

    if start_year is not None and not isinstance(year, int):
        # Only apply start_year if you didn't already pin to a specific year
        params["start_year"] = f"{start_year}-01-01"
        where_clauses.append("a.date >= %(start_year)s")

    # Default WHERE to true if no filters provided
    where_sql = " AND ".join(where_clauses) if where_clauses else "1=1"

    sql = f"""
        SELECT
            *
        FROM ibes.ptgdet a
        WHERE
            a.ANNDATS >= DATE '2024-01-01' AND
            a.ANNDATS <= DATE '2024-12-31'
        ORDER BY a.ANNDATS;
    """

    df = db.raw_sql(sql, params=params)
    return df



In [ ]:
# Load IBES data
ibes = get_ibes(db)

In [57]:
print(len(ibes))
ibes.head()

113453


,ticker,ncusip,oftic,cname,actdats,estimid,alysnam,horizon,value,estcur,curr,amaskcd,usfirm,measure,acttims,anndats,anntims,__idx
0,00VQ,N3167Y10,RACE,FERRARI,2024-01-01,EQUISIGH,TALSANIA P,12,343.623,USD,EUR,200938.0,1,PTG,11:13:10,2024-01-01,11:01:00,0
1,0312,64119V30,NTST,NETSTREIT US,2024-01-01,WOLFE,"ROSIVACH, CFA A",12,22.0,USD,USD,155538.0,1,PTG,17:24:31,2024-01-01,17:19:00,1
3,03LX,81730H10,S,SENTINELONE,2024-01-01,EQUISIGH,TALSANIA P,12,25.86,USD,USD,200938.0,1,PTG,10:51:45,2024-01-01,08:35:00,3
244,AMD,00790310,AMD,AMD,2024-01-01,STIFEL,ROY R,12,170.0,USD,USD,108350.0,1,PTG,21:02:50,2024-01-01,20:18:00,244
245,BAYR,07273030,BAYRY,BAYER,2024-01-01,EQUISIGH,TALSANIA P,12,50.18,USD,USD,200938.0,1,PTG,21:59:25,2024-01-01,09:42:00,245


In [60]:
# Change data format
ibes['horizon'] = pd.to_numeric(ibes['horizon'], errors='coerce')
ibes['anndats'] = pd.to_datetime(ibes['anndats'], format='%Y-%m-%d', errors='coerce')

# Change column name
ibes.rename(columns={'cusip': 'ncusip'}, inplace=True)

In [30]:
# Only keep US firms and only 12m price target forecasts
ibes = ibes.loc[(ibes["usfirm"] == 1) & (ibes["horizon"] == 12)]
ibes.head()

,ticker,cusip,oftic,cname,actdats,estimid,alysnam,horizon,value,estcur,curr,amaskcd,usfirm,measure,acttims,anndats,anntims
0,00VQ,N3167Y10,RACE,FERRARI,2024-01-01,EQUISIGH,TALSANIA P,12,343.623,USD,EUR,200938.0,1,PTG,11:13:10,2024-01-01,11:01:00
1,0312,64119V30,NTST,NETSTREIT US,2024-01-01,WOLFE,"ROSIVACH, CFA A",12,22.0,USD,USD,155538.0,1,PTG,17:24:31,2024-01-01,17:19:00
3,03LX,81730H10,S,SENTINELONE,2024-01-01,EQUISIGH,TALSANIA P,12,25.86,USD,USD,200938.0,1,PTG,10:51:45,2024-01-01,08:35:00
244,AMD,00790310,AMD,AMD,2024-01-01,STIFEL,ROY R,12,170.0,USD,USD,108350.0,1,PTG,21:02:50,2024-01-01,20:18:00
245,BAYR,07273030,BAYRY,BAYER,2024-01-01,EQUISIGH,TALSANIA P,12,50.18,USD,USD,200938.0,1,PTG,21:59:25,2024-01-01,09:42:00


In [ ]:
##############################################

# Now IBES - CRSP

##############################################

In [31]:
import os
import pandas as pd

def get_ibes_link(db, permno_list=None, year=None, start_year=2000):
    """
    Get IBES linking table to link IBES data to CRSP.

    Parameters
    ----------
    db : object
        WRDS connection with a .raw_sql() method.
    permno_list : list[int] | None
        PERMNOs to filter on. If None, returns all PERMNOs.
    year : int | 'all' | None
        If int, restricts to that calendar year (YYYY).
        If 'all' or None, no single-year filter is applied.
    start_year : int | None
        If set, restricts to dates >= January 1 of start_year.

    Returns
    -------
    pandas.DataFrame
        Columns (typical): ticker, permno, ncusip, sdate, edate, score
    """
    # Ensure required directories exist (optional)
    os.makedirs('data', exist_ok=True)
    os.makedirs('data/crsp', exist_ok=True)

    # Build predicates and parameters safely
    where_clauses = []
    params = {}

    if permno_list:
        # Make a list of positional parameters: %(p0)s, %(p1)s, ...
        ph = []
        for i, p in enumerate(permno_list):
            key = f"p{i}"
            params[key] = int(p)  # ensure int
            ph.append(f"%({key})s")
        where_clauses.append(f"a.permno IN ({', '.join(ph)})")

    if isinstance(year, int):
        params["y_start"] = f"{year}-01-01"
        params["y_end"]   = f"{year}-12-31"
        where_clauses.append("a.date BETWEEN %(y_start)s AND %(y_end)s")

    if start_year is not None and not isinstance(year, int):
        # Only apply start_year if you didn't already pin to a specific year
        params["start_year"] = f"{start_year}-01-01"
        where_clauses.append("a.date >= %(start_year)s")

    # Default WHERE to true if no filters provided
    where_sql = " AND ".join(where_clauses) if where_clauses else "1=1"

    sql = f"""
        SELECT
            *
        FROM wrdsapps.ibcrsphist a
    """

    df = db.raw_sql(sql, params=params)
    return df


In [48]:
# Load ibes link data
ibes_link = get_ibes_link(db)
ibes_link.head(20)

,ticker,permno,ncusip,sdate,edate,score
0,0000,14471,87482X10,2014-02-20,2016-08-31,1.0
1,0001,14392,26878510,2014-02-20,2019-05-22,1.0
2,0001,<NA>,<NA>,2019-06-20,2023-12-31,6.0
3,0004,14418,02504D10,2014-02-20,2018-08-24,1.0
4,000R,14378,14163310,2014-02-20,2020-02-10,1.0
5,000V,14423,15117E10,2014-03-20,2016-03-22,1.0
6,000V,14423,28249U10,2016-04-14,2024-01-07,1.0
7,000V,14423,28249U20,2024-01-18,2024-04-10,1.0
8,000V,<NA>,<NA>,2024-04-18,<NA>,6.0
9,000Y,14436,90400D10,2014-03-20,2024-12-31,1.0


In [ ]:
# Change data format
ibes_link['sdate'] = pd.to_datetime(ibes_link['sdate'], format='%Y-%m-%d', errors='coerce')
ibes_link['edate'] = pd.to_datetime(ibes_link['edate'], format='%Y-%m-%d', errors='coerce')
ibes_link['edate'] = pd.to_datetime(ibes_link['edate'], errors='coerce').fillna(pd.Timestamp.today().normalize()) # Fill NA with date of today
ibes_link.head(25)

,ticker,permno,ncusip,sdate,edate,score
0,0000,14471,87482X10,2014-02-20,2016-08-31,1.0
1,0001,14392,26878510,2014-02-20,2019-05-22,1.0
2,0001,<NA>,<NA>,2019-06-20,2023-12-31,6.0
3,0004,14418,02504D10,2014-02-20,2018-08-24,1.0
4,000R,14378,14163310,2014-02-20,2020-02-10,1.0
5,000V,14423,15117E10,2014-03-20,2016-03-22,1.0
6,000V,14423,28249U10,2016-04-14,2024-01-07,1.0
7,000V,14423,28249U20,2024-01-18,2024-04-10,1.0
8,000V,<NA>,<NA>,2024-04-18,2025-09-26,6.0
9,000Y,14436,90400D10,2014-03-20,2024-12-31,1.0


In [68]:
# Merge IBES with link table. Take into account sdate and edate
ibes['__idx'] = ibes.index
ibes_merged = ibes.merge(ibes_link[['ncusip','permno','sdate','edate']], on='ncusip', how='left')

# keep only exact link-window matches
exact = ibes_merged.loc[
    ibes_merged['permno'].notna()
    & (ibes_merged['anndats'] >= ibes_merged['sdate'])
    & (ibes_merged['anndats'] <= ibes_merged['edate'])
].copy()

exact = exact.sort_values(['__idx']).drop_duplicates('__idx', keep='first')


In [70]:
print(len(exact))
exact.head(20)

108483


,ticker,ncusip,oftic,cname,actdats,estimid,alysnam,horizon,value,estcur,...,amaskcd,usfirm,measure,acttims,anndats,anntims,__idx,permno,sdate,edate
0,00VQ,N3167Y10,RACE,FERRARI,2024-01-01,EQUISIGH,TALSANIA P,12,343.623,USD,...,200938.0,1,PTG,11:13:10,2024-01-01,11:01:00,0,15735,2016-01-14,2024-12-31
1,0312,64119V30,NTST,NETSTREIT US,2024-01-01,WOLFE,"ROSIVACH, CFA A",12,22.0,USD,...,155538.0,1,PTG,17:24:31,2024-01-01,17:19:00,1,19601,2020-09-17,2024-12-31
2,03LX,81730H10,S,SENTINELONE,2024-01-01,EQUISIGH,TALSANIA P,12,25.86,USD,...,200938.0,1,PTG,10:51:45,2024-01-01,08:35:00,3,21415,2021-08-19,2024-12-31
3,AMD,00790310,AMD,AMD,2024-01-01,STIFEL,ROY R,12,170.0,USD,...,108350.0,1,PTG,21:02:50,2024-01-01,20:18:00,244,61241,1976-01-15,2024-12-31
5,CCC2,17296742,C,CITIGROUP,2024-01-01,WHEAT,MAYO M,12,70.0,USD,...,5730.0,1,PTG,12:44:04,2024-01-01,12:03:00,246,70519,2011-05-19,2024-12-31
6,CHH2,16990510,CHH,CHOICE HTL INTL,2024-01-01,JEFFEREG,KATZ D,12,96.0,USD,...,104841.0,1,PTG,10:44:29,2024-01-01,04:45:00,247,85517,1997-11-20,2024-12-31
7,CRBU,14203810,CRBU,CARIBOU,2024-01-01,BROOKCM,RUSH CANN L,12,26.0,USD,...,187285.0,1,PTG,22:29:39,2024-01-01,22:22:00,248,21707,2021-09-16,2024-12-31
8,CRSP,H1718210,CRSP,CRISPR THERAPEUT,2024-01-01,BROOKCM,RUSH CANN L,12,199.0,USD,...,187285.0,1,PTG,22:06:21,2024-01-01,21:51:00,249,16383,2016-11-17,2024-12-31
9,CSL,14233910,CSL,CARLISLE,2024-01-01,FAHN,BLAIR B,12,345.0,USD,...,190688.0,1,PTG,22:22:26,2024-01-01,22:15:00,250,27334,1978-07-20,2024-12-31
10,CTAS,17290810,CTAS,CINTAS,2024-01-01,EQUISIGH,TALSANIA P,12,159.498,USD,...,200938.0,1,PTG,12:55:03,2024-01-01,11:26:00,251,23660,1983-12-15,2024-12-31
